In [8]:
import numpy as np
import sys
import pickle
from PIL import Image

In [9]:
num_classes=10
image = Image.open('../input_data/tv.jpg')
imageResized = image.resize((32, 32))
imageResized = np.array(imageResized).transpose((2,0,1))
train_images= imageResized.reshape((1, 3, 32, 32))
train_images.shape

(1, 3, 32, 32)

In [10]:
pic1=imageResized.transpose((1,2,0))
img1=Image.fromarray(pic1,'RGB').resize((100,100)).show()
pic1.shape

(32, 32, 3)

In [11]:
def cross_entropy(inputs, labels):
    out_num = labels.shape[0]
    p = np.sum(labels.reshape(1,out_num)*inputs)
    loss = -np.log(p)
    return loss

In [12]:
class Net:
    def __init__(self):
        
        lr = 0.01
        self.layers = []
        self.layers.append(Convolution2D(inputs_channel=3, num_filters=6, kernel_size=5, padding=0, stride=1, learning_rate=lr, name='conv1'))
        self.layers.append(ReLu())
        self.layers.append(Maxpooling2D(pool_size=2, stride=2, name='maxpool2'))
        self.layers.append(Convolution2D(inputs_channel=6, num_filters=16, kernel_size=5, padding=0, stride=1, learning_rate=lr, name='conv3'))
        self.layers.append(ReLu())
        self.layers.append(Maxpooling2D(pool_size=2, stride=2, name='maxpool4'))
        self.layers.append(Convolution2D(inputs_channel=16, num_filters=120, kernel_size=5, padding=0, stride=1, learning_rate=lr, name='conv5'))
        self.layers.append(ReLu())
        self.layers.append(Flatten())
        self.layers.append(FullyConnected(num_inputs=120, num_outputs=84, learning_rate=lr, name='fc6'))
        self.layers.append(ReLu2())
        self.layers.append(FullyConnected(num_inputs=84, num_outputs=10, learning_rate=lr, name='fc7'))
#         self.layers.append(Softmax())
        self.lay_num = len(self.layers)

    def train(self, training_data,batch_size):
            x=training_data[0]
            g=0
            for l in range(self.lay_num-1):
                output = self.layers[l].forward(x)
                x = output
            output = self.layers[self.lay_num-1].gaussian_forward(x)
            print(np.argmax(output))

In [13]:
class Convolution2D:

    def __init__(self, inputs_channel, num_filters, kernel_size, padding, stride, learning_rate, name):
        # weight size: (F, C, K, K)
        # bias size: (F) 
        self.F = num_filters
        self.K = kernel_size
        self.C = inputs_channel

        self.weights = np.zeros((self.F, self.C, self.K, self.K))
        self.bias = np.zeros((self.F, 1))
        for i in range(0,self.F):
            self.weights[i,:,:,:] = np.random.normal(loc=0, scale=np.sqrt(1./(self.C*self.K*self.K)), size=(self.C, self.K, self.K))

        self.p = padding
        self.s = stride
        self.lr = learning_rate
        self.name = name

    def zero_padding(self, inputs, size):
        w, h = inputs.shape[0], inputs.shape[1]
        new_w = 2 * size + w
        new_h = 2 * size + h
        out = np.zeros((new_w, new_h))
        out[size:w+size, size:h+size] = inputs
        return out

    def forward(self, inputs):
        # input size: (C, W, H)
        # output size: (N, F ,WW, HH)
        C = inputs.shape[0]
        W = inputs.shape[1]+2*self.p
        H = inputs.shape[2]+2*self.p
        self.inputs = np.zeros((C, W, H))
        for c in range(inputs.shape[0]):
            self.inputs[c,:,:] = self.zero_padding(inputs[c,:,:], self.p)
        WW = (W - self.K)//self.s + 1
        HH = (H - self.K)//self.s + 1
        feature_maps = np.zeros((self.F, WW, HH))
        for f in range(self.F):
            for w in range(WW):
                for h in range(HH):
                    feature_maps[f,w,h]=np.sum(self.inputs[:,w:w+self.K,h:h+self.K]*self.weights[f,:,:,:])+self.bias[f]
        pic1=feature_maps.transpose((1,2,0))
        print(pic1.shape)
        img1=Image.fromarray(pic1,'RGB').resize((312,312)).show()
        return feature_maps

    def feed(self, weights, bias):
        self.weights = weights
        self.bias = bias

class Maxpooling2D:

    def __init__(self, pool_size, stride, name):
        self.pool = pool_size
        self.s = stride
        self.name = name

    def forward(self, inputs):
        self.inputs = inputs
        C, W, H = inputs.shape
        new_width = (W - self.pool)//self.s + 1
        new_height = (H - self.pool)//self.s + 1
        out = np.zeros((C, new_width, new_height))
        for c in range(C):
            for w in range(W//self.s):
                for h in range(H//self.s):
                    out[c, w, h] = np.max(self.inputs[c, w*self.s:w*self.s+self.pool, h*self.s:h*self.s+self.pool])
        pic1=out.transpose((1,2,0))
        print(pic1.shape)
        img1=Image.fromarray(pic1,'RGB').resize((312,312)).show()
        return out

    def extract(self):
        return 
    
class FullyConnected:

    def __init__(self, num_inputs, num_outputs, learning_rate, name):
        self.weights = 0.01*np.random.rand(num_inputs, num_outputs)
        self.bias = np.zeros((num_outputs, 1))
        self.lr = learning_rate
        self.name = name

    def forward(self, inputs):
        self.inputs = inputs
        k=np.dot(self.inputs, self.weights) + self.bias.T
        print(k.shape)
        return k

    def feed(self, weights, bias):
        self.weights = weights
        self.bias = bias
        
    def gaussian_forward(self,inputs):
        self.inputs = inputs
        k=np.sum((self.inputs-self.weights.T)*(self.inputs-self.weights.T),axis=1)+self.bias.T
        print(k.shape)
        return k

class Flatten:
    def __init__(self):
        pass
    def forward(self, inputs):
        self.C, self.W, self.H = inputs.shape
        k=inputs.reshape(1, self.C*self.W*self.H)
        print(k.shape)
        return k
    def extract(self):
        return

class ReLu:
    def __init__(self):
        pass
    def forward(self, inputs):
        self.inputs = inputs
        ret = inputs.copy()
        ret[ret < 0] = 0
        pic1=ret.transpose((1,2,0))
        img1=Image.fromarray(pic1,'RGB').resize((312,312)).show()
        return ret
    def extract(self):
        returnforward

class ReLu2:
    def __init__(self):
        pass
    def forward(self, inputs):
        self.inputs = inputs
        ret = inputs.copy()
        ret[ret < 0] = 0
        print(ret.shape)
        return ret
    def extract(self):
        return

class Softmax:
    def __init__(self):
        pass
    def forward(self, inputs):
        exp = np.exp(inputs, dtype=np.float)
        self.out = exp/np.sum(exp)
        print(self.out.shape)
        return self.out
    def extract(self):
        return

In [14]:
net = Net()
print('Training Lenet......')
net.train(train_images,10)

Training Lenet......
(28, 28, 6)
(14, 14, 6)
(10, 10, 16)
(5, 5, 16)
(1, 1, 120)
(1, 120)
(1, 84)
(1, 84)
(1, 10)
8
